In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
import datetime
import time
import os
import json

from src.data import NSynthDataset
from src.utils import print_and_log

/home/ec2-user/.local/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
class Classifier(nn.Module):
    def __init__(self, n_classes, h_dim=128):
        super(Classifier, self).__init__()
        
        # (1, 128, 251)
        self.conv1 = nn.Conv2d(1, 32, 4, padding=(1, 4), stride=(2, 2))
        self.conv2 = nn.Conv2d(32, 64, 4, padding=1, stride=(2, 2))
        self.conv3 = nn.Conv2d(64, 128, 4, padding=1, stride=(2, 2))
        self.conv4 = nn.Conv2d(128, 256, 4, padding=1, stride=(2, 2))
        self.conv5 = nn.Conv2d(256, 512, 4, padding=1, stride=(2, 2))

        self.fc_enc = nn.Linear(512*4*8, h_dim, bias=True)
        self.fc_out = nn.Linear(h_dim, n_classes, bias=True)
        
        # (1, 40, 126)
#         self.conv1 = nn.Conv2d(1, 32, 4, padding=1, stride=(2, 2))
#         self.conv2 = nn.Conv2d(32, 64, 4, padding=1, stride=(2, 2))
#         self.conv3 = nn.Conv2d(64, 128, 4, padding=1, stride=(2, 2))
#         self.conv4 = nn.Conv2d(128, 256, 4, padding=1, stride=(2, 2))
        
#         self.fc_enc = nn.Linear(3584, h_dim, bias=True)
#         self.fc_dec = nn.Linear(h_dim, 3584, bias=True)

#         self.deconv1 = nn.ConvTranspose2d(256, 128, 3, stride=2)
#         self.deconv2 = nn.ConvTranspose2d(128, 64, (2, 3), stride=2)
#         self.deconv3 = nn.ConvTranspose2d(64, 32, (2, 3), stride=2)
#         self.deconv4 = nn.ConvTranspose2d(32, 1, 2, stride=2)
        
    def encode(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
    
        x = x.view(x.size(0), -1)
        return F.relu(self.fc_enc(x))
        
    def forward(self, x):
        z = self.encode(x)
        outputs = self.fc_out(z)
        return outputs

In [3]:
hparams = {
    'feature_type' : 'mel',
    'instrument_source' : [0],
    'scaling' : 'normalize',
    'n_epochs' : 25,
    'batch_size' : 32,
    'lr' : 0.001,
    'hidden_dim' : 128,
    'display_iters' : 10,
    'val_iters' : 100,
    'n_val_samples' : 1000, 
    'n_early_stopping' : 5 # stop if validation doesn't improve after this number of validation cycles
}

In [4]:
train_dataset = NSynthDataset(
    'music-ml-gigioli', 
    'data/nsynth/nsynth-train', 
    instrument_source=hparams['instrument_source'], 
    feature_type=hparams['feature_type'],
    scaling=hparams['scaling'],
    include_meta=True
)

val_dataset = NSynthDataset(
    'music-ml-gigioli', 
    'data/nsynth/nsynth-valid', 
    instrument_source=hparams['instrument_source'], 
    feature_type=hparams['feature_type'],
    scaling=hparams['scaling'],
    include_meta=True
)

In [5]:
class_ctr = Counter([x['instrument_family_str'] for x in train_dataset.meta.values()])
class_dict = dict(enumerate(sorted(class_ctr)))
inv_class_dict = dict([(v, k) for k, v in class_dict.items()])
class_weights = np.array([max(class_ctr.values())/class_ctr[class_dict[i]] for i in range(len(class_dict))])

In [14]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=True)

In [15]:
model = Classifier(n_classes=len(class_dict), h_dim=hparams['hidden_dim'])

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'])

In [17]:
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d-%H%M%S')
results_dir = 'train_results/classifier/{}'.format(timestamp)
os.makedirs(results_dir)

with open(os.path.join(results_dir, 'hparams.json'), 'w') as fp:
    json.dump(hparams, fp)

log_file = os.path.join(results_dir, 'train_log.txt')
log = open(log_file, 'w')
log.close()

In [ ]:
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32))

ckpt_path = None
best_loss = 1e10
since_best = 0
done = False
for epoch in range(hparams['n_epochs']):
    if done:
        break
        
    itr = 0
    for batch in train_dataloader:
        if done:
            break
            
        itr += 1
        features = batch[0].unsqueeze(1)
        labels = torch.tensor([inv_class_dict[i] for i in batch[1]['instrument_family_str']], dtype=torch.long)
        outputs = model(features)
        
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (itr % hparams['display_iters'] == 0) or (itr == 1):
            acc = (outputs.argmax(-1) == labels).float().mean()
            print_and_log('[{}, {:5d}] loss : {:.4f}, acc : {:.4f}'.format(epoch, itr, loss.item(), acc.item()), log_file)
            
        if itr % hparams['val_iters'] == 0:
            val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=True)
            
            model.eval()
            
            ct, i, val_loss, val_acc = 0, 0, 0.0, 0.0
            for batch in val_dataloader:
                i += 1
                ct += batch[0].size(0)
                features = batch[0].unsqueeze(1)
                labels = torch.tensor([inv_class_dict[i] for i in batch[1]['instrument_family_str']], dtype=torch.long)
                outputs = model(features)
                
                loss = loss_fn(outputs, labels)
                acc = (outputs.argmax(-1) == labels).float().mean()
                
                val_loss += (loss.item() - val_loss)/i
                val_acc += (acc.item() - val_acc)/i
                
                if ct >= hparams['n_val_samples']:
                    break
                
            print_and_log('Val - loss : {:.4f}, acc : {:.4f}'.format(val_loss, val_acc), log_file)
            
            if val_loss < best_loss:
                since_best = 0
                best_loss = val_loss
                
                if ckpt_path:
                    os.remove(ckpt_path)
                ckpt_path = os.path.join(results_dir, 'model-{}.weights'.format(itr))
                torch.save(model.state_dict(), ckpt_path)
                print_and_log('Weights saved in {}'.format(ckpt_path), log_file)
            else:
                since_best += 1
                if since_best >= hparams['n_early_stopping']:
                    done = True
                    print_and_log('Early stopping... training complete', log_file)
            
            model.train()

[0,     1] loss : 2.3158, acc : 0.0625
[0,    10] loss : 2.3276, acc : 0.0312
[0,    20] loss : 2.1975, acc : 0.2812
[0,    30] loss : 2.7643, acc : 0.1250
[0,    40] loss : 2.2617, acc : 0.2812
[0,    50] loss : 2.1652, acc : 0.1250
[0,    60] loss : 2.2949, acc : 0.0312
[0,    70] loss : 2.2701, acc : 0.0312
[0,    80] loss : 2.4682, acc : 0.0312
[0,    90] loss : 2.1815, acc : 0.1250
[0,   100] loss : 2.2860, acc : 0.2188
Val - loss : 2.2783, acc : 0.1318
Weights saved in train_results/classifier/20200428-220200/model-100.weights
[0,   110] loss : 2.2304, acc : 0.3125
[0,   120] loss : 3.9148, acc : 0.0625
[0,   130] loss : 2.1717, acc : 0.4688
[0,   140] loss : 2.3052, acc : 0.1875
[0,   150] loss : 2.0743, acc : 0.1875
[0,   160] loss : 2.0673, acc : 0.4062
[0,   170] loss : 3.5514, acc : 0.4062
[0,   180] loss : 1.9452, acc : 0.4062
[0,   190] loss : 2.0416, acc : 0.2812
[0,   200] loss : 1.8759, acc : 0.2812
Val - loss : 1.9569, acc : 0.2939
Weights saved in train_results/classi